In [1]:
from collections import Counter
import gc
import numpy as np
from typing import List, Dict, Iterable
from itertools import chain

import fitz
from fitz.utils import getColor
from par_seg import get_spans_lines, gpk
from par_seg import set_dic_par, sort_byreading
from par_seg import get_par_text

COLORS = fitz.utils.getColorList()
NOTWHITE = [i for i in COLORS if 'WHITE' not in i]
np.random.shuffle(NOTWHITE)

FORBIDDEN = [
    'Este documento pode ser',
    'Documento assinado digita',
    'Infraestrutura de Chaves',
    'PÁG.',
    'SEÇÃO',
    'pelo código',
]

In [2]:
%%time
fpath = 'pdf/2020_04_30.pdf'
doc = fitz.open(fpath)
gc.collect()

CPU times: user 24.1 ms, sys: 0 ns, total: 24.1 ms
Wall time: 24 ms


20

In [3]:
spans, lines = get_spans_lines(doc, glue_horizon=True)

spans_sorted = sort_byreading(spans)
lines_sorted = sort_byreading(lines)

In [4]:
# GOING TO FILTER THROUGH SOME HEURISTICS!
par_lis = set_dic_par(spans_sorted)
acc = 0
# doc = fitz.open(fpath)
valid_texts = []
for idx, (key, l) in enumerate( gpk( par_lis, 'par').items() ):
    color = getColor(NOTWHITE[idx % len(NOTWHITE)])
    whole_tex = '\n'.join([sp['text'] for sp in l])
    if '\n' not in whole_tex or len(whole_tex) < 40:
        continue
    valid_texts.append(whole_tex)
    acc += 1
    for sp in l:
        page = doc[sp['page']//2]  # voltando ao valor original
        page.drawRect( sp['bbox'], color=color, width=1)
doc.save('pdf/par_spans_filtered.pdf')
print(acc)

355


In [5]:
ps=get_par_text(doc)
assert valid_texts == ps

In [6]:
# # doc = fitz.open(fpath)
# par_lis = set_dic_par(lines)
# for idx, (key, l) in enumerate( gpk( par_lis, 'par').items() ):
#     color = getColor(NOTWHITE[idx % len(NOTWHITE)])
#     for sp in l:
#         page = doc[sp['page']//2]
#         page.drawRect( sp['bbox'], color=color, width=1)
# doc.save('pdf/par_lines2.pdf')

# par_lis = set_dic_par(spans_sorted)
# for idx, (key, l) in enumerate( gpk( par_lis, 'par').items() ):
#     color = getColor(NOTWHITE[idx % len(NOTWHITE)])
#     for sp in l:
#         page = doc[sp['page']//2]  # voltando ao valor original
#         page.drawRect( sp['bbox'], color=color, width=1)
# doc.save('pdf/par_spans2.pdf')